In [1]:
import numpy as np
import torch
import pandas as pd

import matplotlib.pyplot as plt

from toy_datasets.rotational_dataset import make_dataset
from fourier_utils import fourier_transform_point_function, inverse_transform, Complex

In [2]:
dataset = make_dataset(30, 120)

In [3]:
dataset

,vertex_id,x,y,theta,h,output_forces,graph_id
0,0,0.104180,-0.668324,6.130552,-1.979291,19.146745,0
1,1,-1.708887,0.453158,5.626084,1.792478,-11.831672,0
2,2,-1.249395,-1.273291,1.556247,1.642766,-20.562209,0
3,3,-1.531083,-1.153175,2.038291,-0.691341,-25.997470,0
4,4,-0.318630,-1.100289,0.957461,0.482672,3.949301,0
...,...,...,...,...,...,...,...
3595,115,0.874069,0.293882,0.141910,-2.851216,5.388856,29
3596,116,-1.096148,-1.041422,2.640851,1.264794,11.439627,29
3597,117,-0.321533,0.155805,4.664408,0.855477,17.526319,29
3598,118,-1.240098,-1.515808,2.940352,1.819010,7.136274,29


### Define a pipeline for casting the raw data to a steerable space

In [4]:
fourier_precision = 10

In [5]:
c_outs = []

for row_num, row in enumerate(dataset['theta']):
    c = fourier_transform_point_function(1, row, precision=fourier_precision)
    data = {}
    for i, j in c:
        i = int(i.real)
        data[f'c_{i}'] = j
    c_row = pd.DataFrame(data, index=[row_num])
    c_outs.append(c_row)
coefficients = pd.concat(c_outs)
dataset = dataset.merge(coefficients,
                        left_index=True,
                        right_index=True
                        )

#### Now define a transformer that works on these

In [7]:
class MLP(torch.nn.Module):

     def __init__(self, input_dim, width, output_dim):
        super().__init__()

        layers = [
            torch.nn.Linear(input_dim, width),
            torch.nn.ReLU(),
            torch.nn.Linear(width, output_dim)
        ]
        self.mlp = torch.nn.Sequential(layers)


class BaseTransformer(torch.nn.Module):
    """Abstract class. Need to subclass and replace
     the 'invariant computer' method to fix. You also need to """

    def __init__(self, features_mlp_width, attention_mlp_width, invariant_dimension):
        super().__init__()

        self.invariant_dimension = invariant_dimension

        self.hidden_features_computer = MLP(1, features_mlp_width, 1)
        self.attention_coefficients_computer = MLP(self.invariant_dimension, attention_mlp_width, 1) # Should this actually be an MLP??

    def forward(self, x, y, theta, input_features):
        """
        :param x:
        :return: predicted outputs for every node
        """

        hidden_features = self.hidden_features(input_features)
        pairwise_invariants = self.invariant_computer(x, y, theta)
        attention = self.attention_coefficients_computer(pairwise_invariants)
        outputs = attention @ hidden_features

        return outputs

    @staticmethod
    def invariant_computer(*coords):
        raise(NotImplementedError)

class DistanceOnlyTransformer(BaseTransformer):
    def __init__(self, features_mlp_width, attention_mlp_width):
        super().__init__(features_mlp_width, attention_mlp_width, 1)

    @staticmethod
    def invariant_computer(x, y, theta):
        return np.linalg.norm(x - y)

class RelativePositionTransformer(BaseTransformer):
    def __init__(self, features_mlp_width, attention_mlp_width):
        super().__init__(features_mlp_width, attention_mlp_width, 2)

    @staticmethod
    def invariant_computer(x, y, theta):
        return x - y # What to do here! Need to make this invariant/equivariant?

class FullTransformer(BaseTransformer):
    def __init__(self, features_mlp_width, attention_mlp_width):
        super().__init__(features_mlp_width, attention_mlp_width, 3)

    @staticmethod
    def invariant_computer(x, y, theta):
        return x - y




### Need to test the equivariance/invariance properties